# GridLAB-D Module

* Use `command` to send options
* Command are processed in the order they are received
* Errors are not caught until the start command is used
* Compound commands must be separated

In [1]:
import gridlabd
[gridlabd.command(option) for option in "-W developer session7_1.glm".split()]

[1, 2, 3]

`command` returns the total number of command line options received so far

* Use `start` to begin simulation
* Two possible starts modes
  - `thread`: starts as a separate thread (e.g., server)
  - `wait`: starts in the same thread (e.g., job)

In [2]:
gridlabd.start('wait')

HiPAS GridLAB-D
{'major': 4, 'minor': 3, 'patch': 1, 'build': 220807, 'branch': 'develop_user_manual'}
HiPAS GridLAB-D 4.3.1

Copyright (C) 2008-2017, Battelle Memorial Institute.
Copyright (C) 2016-2022, The Board of Trustees of the Leland Stanford Junior University.
All Rights Reserved.
For additional information, see http://www.gridlabd.us/.

WARNING  [INIT] : a warning message
ERROR    [INIT] : an error message


5

# Global access

Get a list of all the globals

In [3]:
sorted(gridlabd.get('globals'))[:5]

['MONTH', 'YEAR', 'allow_reinclude', 'allow_variant_aggregates', 'autoclean']

Get a global value (always returns a `str` value)

In [4]:
gridlabd.get_global('YEAR')

'2022'

Values will be formatted according to the GridLAB-D data type (not Python)

In [5]:
gridlabd.get_global('starttime')

'2022-07-01 00:00:00 PDT'

Setting a value must use the GridLAB-D data type (not Python) and always returns the old value (if any).

In [6]:
gridlabd.set_global('starttime',f'2022-07-02 00:00:00 PDT')

'2022-07-01 00:00:00 PDT'

# Class access

Classes are defined in modules (if any) and/or by a `class` declaration in the GLM file.

A list of modules and classes can be obtained using the `get` function.

In [7]:
gridlabd.get('modules')

[]

In [8]:
gridlabd.get('classes')[:5]

['example']

Get a class definition (returns a Python `dict` object)
* Includes the details of properties as `dict` objects.

In [9]:
gridlabd.get_class('example')

{'class.object_size': 822,
 'class.trl': 0,
 'profiler.numobjs': 6,
 'profiler.clocks': 0,
 'profiler.count': 0,
 'price': {'type': 'double', 'access': 'PUBLIC', 'unit': '$/MW'},
 'real_power': {'type': 'double', 'access': 'PUBLIC', 'unit': 'MW'},
 'power_factor': {'type': 'double', 'access': 'PUBLIC'},
 'power': {'type': 'complex', 'access': 'PUBLIC', 'unit': 'MVA'},
 'counter': {'type': 'int32', 'access': 'PUBLIC'},
 'label': {'type': 'char32', 'access': 'PUBLIC'},
 'load': {'type': 'enduse', 'access': 'PUBLIC'},
 'is_ok': {'type': 'bool', 'access': 'PUBLIC'},
 'last_change': {'type': 'timestamp', 'access': 'PUBLIC'}}

# Object access

A list of objects is also available using the `get` function.

In [10]:
gridlabd.get('objects')

['main', 'sub_1', 'sub_2', 'sub_3', 'sub_4', 'sub_5']

Objects are accessed using the object name (or `class`:`id` for anonymous objects).

In [11]:
gridlabd.get_object('sub_5')

{'id': 5,
 'name': 'sub_5',
 'class': 'example',
 'parent': 'main',
 'rank': 0,
 'clock': '2022-07-01 00:00:00 PDT',
 'schedule_skew': 0,
 'rng_state': 112371602,
 'heartbeat': 0,
 'guid': '7eba720f961fa19d',
 'price': '-0.521325 $/MW',
 'real_power': '+1.66893 MW',
 'power_factor': '+0.935197',
 'power': '+0+0i MVA',
 'counter': '0',
 'label': '""',
 'load': '+0+0j',
 'is_ok': 'FALSE',
 'last_change': 'INIT'}

# Datetime values

GridLAB-D `timestamp` values can be converted to Python `datetime` using the Python `datetime` module.

In [12]:
import datetime
datetime.datetime.strptime(gridlabd.get_global('stoptime'),"%Y-%m-%d %H:%M:%S %Z")

datetime.datetime(2022, 7, 28, 0, 0)

GridLAB-D `timestamp` values are timezone aware. To make Python `datetime` include the timezone, use `pytz`.

In [13]:
import pytz
local_tz = pytz.timezone('America/Los_Angeles')

In [14]:
local_time = datetime.datetime.strptime(gridlabd.get_global('stoptime'),"%Y-%m-%d %H:%M:%S %Z")
local_tz.localize(local_time)

datetime.datetime(2022, 7, 28, 0, 0, tzinfo=<DstTzInfo 'America/Los_Angeles' PDT-1 day, 17:00:00 DST>)

**Important note**: `pytz` does ***not*** apply historical TZ rules.

# Real values

GridLAB-D `double` values are equivalent to Python `float` values.

In [15]:
value=gridlabd.get_value('sub_1','real_power')
value

'+3.10775 MW'

You can define your own accessor to obtain the `float` value

In [16]:
def get_double(x): return float(x.split()[0])
get_double(value)

3.10775

You can also define an accessor to obtain the units

In [17]:
def get_unit(x): return x.split()[1]
get_unit(value)

'MW'

GridLAB-D provides a unit conversion utility to handle non-standard units

In [18]:
gridlabd.convert_unit(get_double(value),get_unit(value),"kW")

3107.75

You can define an accessor to handle the unit conversion automatically

In [19]:
def get_double_unit(x,u): return gridlabd.convert_unit(*(get_double(x),get_unit(x)),u)
get_double_unit(value,'W')

3107750.0

# Setting values

Setting a value requires a string be provided. The type conversion is handled by GridLAB-D.

In [20]:
gridlabd.set_value('sub_1','power_factor','0.95'), gridlabd.get_value('sub_1','real_power')

('+0.922519', '+3.10775 MW')

The unit conversion is also handled by GridLAB-D.

In [21]:
gridlabd.set_value('sub_1','real_power','2500 kW'), gridlabd.get_value('sub_1','real_power')

('+3.10775 MW', '+2.5 MW')

In [22]:
value = 3.0e6
gridlabd.set_value('sub_1','real_power',f"{value} W"), gridlabd.get_value('sub_1','real_power')

('+2.5 MW', '+3 MW')

Mismatched units will cause an error message. The value is still accepted, but it's not converted.

In [23]:
gridlabd.set_value('sub_1','real_power','10 V'), gridlabd.get_value('sub_1','real_power')

ERROR    [INIT] : could not convert units from V to MW, mismatched constant values
ERROR    [INIT] : convert_to_double(const char *buffer='10 V', void *data=0x0x7ff59b909d60, PROPERTY *prop={name='real_power',...}): unit conversion failed


('+3 MW', '+10 MW')

# Complex Values

Complex values are written using the complex notation of the most recent write operation

In [24]:
gridlabd.get_value('sub_1','power')

'+0+0i MVA'

In [25]:
gridlabd.set_value('sub_1','power','3+2d MVA'),gridlabd.get_value('sub_1','power')

('+0+0i MVA', '+3+2d MVA')

You can create you own accessor, but beware of the limitation of the `complex` Python type.

In [26]:
def get_complex(x): return complex(x.split()[0].replace('i','j'))

In [27]:
get_complex(gridlabd.get_value('sub_1','power'))

(2.998172481057287+0.1046984901075029j)

Unit conversion works for complex values as well.

In [28]:
gridlabd.convert_unit(get_complex(gridlabd.get_value('sub_1','power')).real,
                      get_unit(gridlabd.get_value('sub_1','power')),
                      "kW")

2998.172481057287

# Integer Values

All three integer types are accessible and correspond to the Python `int` type.

In [29]:
gridlabd.get_value('main','counter')

'0'

Your own accessor is very simple to implement

In [30]:
def get_integer(x): return int(x)
get_integer(gridlabd.get_value('main','counter'))

0

Globals named `SEQ_<str>` are integers and they have a special syntax and behavior

In [31]:
gridlabd.get_global('SEQ_A:INIT')

'0'

In [32]:
gridlabd.get_global('SEQ_A:INC'), gridlabd.get_global('SEQ_A:INC'), gridlabd.get_global('SEQ_A:INC')

('1', '2', '3')

In [33]:
gridlabd.set_global('SEQ_A','0'), gridlabd.get_global('SEQ_A')

('3', '0')

# Boolean values

GridLAB-D Boolean values are the strings "TRUE" and "FALSE".

In [34]:
gridlabd.get_value('main','is_ok')

'FALSE'

The strings "0" and "1" are accepted too.

In [35]:
gridlabd.set_value('main','is_ok','TRUE'), \
    gridlabd.set_value('main','is_ok','0'), \
    gridlabd.set_value('main','is_ok','1'), \
    gridlabd.set_value('main','is_ok','FALSE')


('FALSE', 'TRUE', 'FALSE', 'TRUE')

Your accessor should check whether the string is valid a GridLAB-D Boolean.

In [36]:
def get_boolean(x):
    if x == "TRUE":
        return True
    if x == "FALSE":
        return False
    raise Exception("invalid Boolean value")
get_boolean("TRUE"), get_boolean("FALSE")

(True, False)

In [37]:
try:
    get_boolean("0")
except Exception as err:
    print(err)

invalid Boolean value


# Strings

String usually are returned in enclosing double quotes.

In [38]:
gridlabd.get_value('main','label')

'""'

In [39]:
gridlabd.set_value('main','label','example text'), gridlabd.get_value('main','label')

('""', '"example text"')

Your accessor needs to remove the quotes before using the string

In [40]:
def get_string(x): return gridlabd.get_value('main','label').strip('"')

In [41]:
value = get_value('main','label')
get_string(value)

'example text'

# Enduses

`enduse` properties calculate electrical load components using the following parts:
  - `current_fraction`
  - `impedance_fraction`
  - `power_fraction`
  - `power_factor`
  - `power.real`
  - `power.imag`
  - `loadshape`
  
Value returned is always as of the last update to enduse load

In [42]:
gridlabd.get_value('main','load')

'+0+0j'

Changes to enduse load components do not immediately take effect (`enduse` requires a `precommit` event)

In [43]:
gridlabd.set_value('main','load','power.real:1.2'), gridlabd.get_value('main','load')

('+0+0j', '+0+0j')

# Data accessors

Direct access to underlying data is extremely fast but not checked for validity

In [44]:
price = gridlabd.get_property('sub_1','price')
price

140692853661016

Basic data type values use `gridlabd.{get,set}_<TYPE>`:

In [45]:
gridlabd.get_double(price), gridlabd.set_double(price,1.5), gridlabd.get_double(price)

(1.3332922142764208, 1.3332922142764208, 1.5)

In [46]:
power = gridlabd.get_property('sub_1','power')
gridlabd.get_complex(power), gridlabd.set_complex(power,2+1j), gridlabd.get_complex(power)

((2.998172481057287+0.1046984901075029j),
 (2.998172481057287+0.1046984901075029j),
 (2+1j))

In [47]:
counter = gridlabd.get_property('sub_1','counter')
gridlabd.get_int32(counter), gridlabd.set_int32(counter,-1), gridlabd.get_int32(counter)

(0, 0, -1)

In [48]:
is_ok = gridlabd.get_property('sub_1','is_ok')
gridlabd.get_bool(is_ok), gridlabd.set_bool(is_ok,True), gridlabd.get_bool(is_ok)

(False, False, True)

Notes: 
1. Strings do have property accessors; use `{get,set}_value` instead.
2. Timestamp should be accessed using type `int64`.

# Accessor class

You can create a convenience accessor class to handle data types, e.g.,

In [49]:
!cat accessor.py

import gridlabd
class Property:
    properties = {}
    def __init__(self,obj,name,using_type=None,from_class=None):
        if not using_type:
            using_type = gridlabd.get_class(from_class)[name]['type']
        if not obj in self.properties.keys():
            data = {}
            self.properties[obj] = data
        else:
            data = self.properties[obj]
        if not name in data.keys():
            addr = gridlabd.get_property(obj,name)
            data[name] = addr
        else:
            addr = data[name]
        self.addr = gridlabd.get_property(obj,name)
        self.get_value = eval(f"gridlabd.get_{using_type}")
        self.set_value = eval(f"gridlabd.set_{using_type}")

    def get(self):
        return self.get_value(self.addr)

    def set(self,value):
        return self.set_value(self.addr,value)

    def __str__(self):
        return str(self.get_value(self.addr))

    def __repr__(self):
        return f"<gridlabd.property {self.addr}>"




# Accessor class

In [50]:
import accessor

In [51]:
price = accessor.Property('sub_1','price','double')
price, str(price), price.get(), price.set(2.0), price.get()

(<gridlabd.property 140692853661016>, '1.5', 1.5, 1.5, 2.0)

In [52]:
power = accessor.Property('sub_1','power','complex')
power, str(power), power.get(), power.set(3+2j), power.get()

(<gridlabd.property 140692853661040>, '(2+1j)', (2+1j), (2+1j), (3+2j))

In [53]:
counter = accessor.Property('sub_1','counter','int32')
counter, str(counter), counter.get(), counter.set(1), counter.get()

(<gridlabd.property 140692853661064>, '-1', -1, -1, 1)

In [54]:
real_power = accessor.Property('sub_1','price',from_class='example')
real_power, str(real_power), real_power.get(), real_power.set(1.2), real_power.get()

(<gridlabd.property 140692853661016>, '2.0', 2.0, 2.0, 1.2)

# Property Accessor

Accessor for properties does validation

In [55]:
price = gridlabd.property('main','price')

In [56]:
price.get_initial()

'+0 $/MW'

In [57]:
price.get_name()

'price'

In [58]:
price.get_unit()

'$/MW'

In [59]:
price.get_value()

0.0

In [60]:
price.set_value(1.23)

In [61]:
try:
    price.set_value(1)
except Exception as err:
    print(err,file=sys.stderr)

value is not a float


# Model I/O

In [62]:
gridlabd.save('example.json')

33914

In [63]:
import json
with open('example.json','r') as fh:
    json_data = json.load(fh)
json_data.keys()

dict_keys(['application', 'version', 'modules', 'types', 'header', 'classes', 'globals', 'schedules', 'objects'])

In [64]:
json_data['objects'].keys()

dict_keys(['main', 'sub_1', 'sub_2', 'sub_3', 'sub_4', 'sub_5'])

In [65]:
json_data['objects']['main']

{'id': '0',
 'class': 'example',
 'rank': '1',
 'clock': '2022-07-01 00:00:00 PDT',
 'rng_state': '823564440',
 'guid': '661A77357AB4BEDB5B12CB07A5A102FC',
 'flags': '0x100',
 'price': '1.23 $/MW',
 'real_power': '0 MW',
 'power_factor': '0',
 'power': '0+0j MVA',
 'counter': '0',
 'label': 'example text',
 'is_ok': 'FALSE',
 'last_change': 'INIT'}

Other Model I/O functions that only work before the simulation has started:
* `gridlabd.module(name)`
* `gridlabd.add(block,data)`
* `gridlabd.load(filename)`